In [19]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=229bee45fd0fdf36bbe109b23302504bdfe362271c2f6accd14e0466fc1c95d3
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [20]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import zipfile
from PIL import Image
from fpdf import FPDF

In [21]:
# 1. Upload dos arquivos CSV
uploaded = files.upload()

Saving dados (1).csv to dados (1) (2).csv
Saving dados (2).csv to dados (2) (2).csv
Saving dados (3).csv to dados (3).csv


In [22]:
# 2. Criar pastas
pasta_imagens = "/content/imagens_emg"
pasta_pdfs = "/content/pdfs_emg"
os.makedirs(pasta_imagens, exist_ok=True)
os.makedirs(pasta_pdfs, exist_ok=True)

In [23]:
# === Configurações ===
usar_normalizacao = True
taxa_amostragem = 1000
min_duracao_ms = 50

In [24]:
# === Funções auxiliares ===
def media_movel(sinal, janela):
    return np.convolve(sinal, np.ones(janela) / janela, mode='valid')

def normalizar(sinal):
    return (sinal - np.min(sinal)) / (np.max(sinal) - np.min(sinal))


In [25]:

# === Processar cada arquivo ===
for nome_arquivo in uploaded.keys():
    try:
        df = pd.read_csv(nome_arquivo)

        tempo = df.iloc[:, 0]
        emg1 = df.iloc[:, 1]
        emg2 = df.iloc[:, 2]

        if usar_normalizacao:
            emg1 = normalizar(emg1)
            emg2 = normalizar(emg2)

        nome_base = os.path.splitext(nome_arquivo)[0].replace(" ", "_").lower()
        partes = nome_base.split("_")
        movimento = "_".join([p for p in partes if not p.isdigit()])
        numero_dado = ''.join(filter(str.isdigit, nome_base))

        imagens_arquivo = []

        # === 3 imagens: janela 5, 10, 20 ===
        for janela in [5, 10, 20]:
            emg1_f = media_movel(emg1, janela)
            emg2_f = media_movel(emg2, janela)
            tempo_filt = tempo[janela - 1:]

            plt.figure(figsize=(12, 6))

            # EMG1
            plt.subplot(2, 1, 1)
            plt.plot(tempo, emg1, label='EMG1 Original', alpha=0.3)
            plt.plot(tempo_filt, emg1_f, label=f'EMG1 Filtro {janela}', color='blue')
            plt.title(f"{nome_base} – EMG1 – Janela {janela}")
            plt.legend()

            # EMG2
            plt.subplot(2, 1, 2)
            plt.plot(tempo, emg2, label='EMG2 Original', alpha=0.3)
            plt.plot(tempo_filt, emg2_f, label=f'EMG2 Filtro {janela}', color='green')
            plt.title(f"{nome_base} – EMG2 – Janela {janela}")
            plt.legend()

            plt.tight_layout()

            nome_img = f"{movimento}_{numero_dado}_janela_{janela}.png"
            caminho_img = os.path.join(pasta_imagens, nome_img)
            plt.savefig(caminho_img, dpi=300)
            plt.close()

            imagens_arquivo.append(caminho_img)

        # === Imagem 4: geral com janelas 5, 10, 20 ===
        emg1_filtros = {}
        emg2_filtros = {}
        tempo_filtros = {}

        for janela in [5, 10, 20]:
            emg1_f = media_movel(emg1, janela)
            emg2_f = media_movel(emg2, janela)
            tempo_f = tempo[janela - 1:]
            emg1_filtros[janela] = emg1_f
            emg2_filtros[janela] = emg2_f
            tempo_filtros[janela] = tempo_f

        plt.figure(figsize=(12, 6))

        # EMG1
        plt.subplot(2, 1, 1)
        plt.plot(tempo, emg1, label='EMG1 Original', alpha=0.3)
        for j in [5, 10, 20]:
            plt.plot(tempo_filtros[j], emg1_filtros[j], label=f'Filtro {j}')
        plt.title(f"{nome_base} – EMG1 – Todos os Filtros")
        plt.legend()

        # EMG2
        plt.subplot(2, 1, 2)
        plt.plot(tempo, emg2, label='EMG2 Original', alpha=0.3)
        for j in [5, 10, 20]:
            plt.plot(tempo_filtros[j], emg2_filtros[j], label=f'Filtro {j}')
        plt.title(f"{nome_base} – EMG2 – Todos os Filtros")
        plt.legend()

        plt.tight_layout()

        nome_img_geral = f"{movimento}_{numero_dado}_janela_geral.png"
        caminho_img_geral = os.path.join(pasta_imagens, nome_img_geral)
        plt.savefig(caminho_img_geral, dpi=300)
        plt.close()

        imagens_arquivo.append(caminho_img_geral)

        # === Criar PDF com as 4 imagens ===
        pdf = FPDF()
        for img_path in imagens_arquivo:
            pdf.add_page()
            pdf.image(img_path, x=10, y=10, w=190)

        nome_pdf = f"{movimento}_{numero_dado}_graficos.pdf"
        caminho_pdf = os.path.join(pasta_pdfs, nome_pdf)
        pdf.output(caminho_pdf)

        print(f"📄 PDF gerado: {caminho_pdf}")

    except Exception as e:
        print(f"❌ Erro ao processar {nome_arquivo}: {e}")

📄 PDF gerado: /content/pdfs_emg/dados_(1)_(2)_12_graficos.pdf
📄 PDF gerado: /content/pdfs_emg/dados_(2)_(2)_22_graficos.pdf
📄 PDF gerado: /content/pdfs_emg/dados_(3)_3_graficos.pdf


In [26]:
# 7. Compactar os PDFs
caminho_zip = "/content/pdfs_emg.zip"
with zipfile.ZipFile(caminho_zip, 'w') as zipf:
    for root, _, files_in_dir in os.walk(pasta_pdfs):
        for file in files_in_dir:
            full_path = os.path.join(root, file)
            zipf.write(full_path, arcname=os.path.relpath(full_path, pasta_pdfs))

# 8. Download do ZIP com os PDFs
files.download(caminho_zip)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>